# NLP3  - Text PreProcessing 

## TMDB data set

    - Fetching data from TMDB API
    - fetching the genre form APT
    - Updating the Genre ids with genres
    - performing Text preprocesing
    
    API genre 
    https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US
    API TMDB 
    https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page=1
    

In [184]:
import pandas as pd
import requests
import re
import nltk
import spacy

In [157]:
url = "https://api.themoviedb.org/3/movie/top_rated?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US&page="

In [158]:
response = requests.get(url)

### Fetching movie data from the url

In [159]:
temp = pd.DataFrame()
for page in range(1,472):
    url1 = url+str(page)
    response = requests.get(url1)
    temp = temp.append(pd.DataFrame(response.json()['results']))

In [160]:
temp.head(10)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/5hNcsnMkwU2LknLoru73c76el3z.jpg,"[35, 18, 10749]",19404,hi,दिलवाले दुल्हनिया ले जायेंगे,"Raj is a rich, carefree, happy-go-lucky second...",29.160,/2CAL2433ZeIihfX1Hb2139CX0pW.jpg,1995-10-20,Dilwale Dulhania Le Jayenge,False,8.7,3317
1,False,/iNh3BivHyg5sQRPP1KOkzguEX0H.jpg,"[18, 80]",278,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,59.217,/q6y0Go1tsGEsmtFryDOJo3dEmqu.jpg,1994-09-23,The Shawshank Redemption,False,8.7,20400
2,False,/rSPw7tgCH9c6NqICZef4kZjFOQ5.jpg,"[18, 80]",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",68.697,/eEslKSwcqmiNS6va24Pbxf2UKmJ.jpg,1972-03-14,The Godfather,False,8.7,15252
3,False,/jtAI6OJIWLWiRItNSZoWjrsUtmi.jpg,[10749],724089,en,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,18.105,/x5o8cLZfEXMoZczTYWLrUo1P7UJ.jpg,2020-07-31,Gabriel's Inferno Part II,False,8.6,1365
4,False,/loRmRzQXZeqG78TqZuyvSlEQfZb.jpg,"[18, 36, 10752]",424,en,Schindler's List,The true story of how businessman Oskar Schind...,37.991,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,1993-11-30,Schindler's List,False,8.6,12182
5,False,/w2uGvCpMtvRqZg6waC1hvLyZoJa.jpg,[10749],696374,en,Gabriel's Inferno,An intriguing and sinful exploration of seduct...,16.430,/oyG9TL7FcRP4EZ9Vid6uKzwdndz.jpg,2020-05-29,Gabriel's Inferno,False,8.6,2176
6,False,/poec6RqOKY9iSiIUmfyfPfiLtvB.jpg,"[18, 80]",240,en,The Godfather: Part II,In the continuing saga of the Corleone crime f...,47.142,/v3KCBeX0CBlZnHZndimm7taYqwo.jpg,1974-12-20,The Godfather: Part II,False,8.6,9179
7,False,/fQq1FWp1rC89xDrRMuyFJdFUdMd.jpg,"[10749, 35]",761053,en,Gabriel's Inferno Part III,The final part of the film adaption of the ero...,30.800,/qtX2Fg9MTmrbgN1UUvGoCsImTM8.jpg,2020-11-19,Gabriel's Inferno Part III,False,8.6,926
8,False,/1EAxNqdkVnp48a7NUuNBHGflowM.jpg,"[16, 28, 878, 18]",283566,ja,シン・エヴァンゲリオン劇場版:||,"In the aftermath of the Fourth Impact, strande...",172.303,/g7xEBLb5hOTPpjQxIMrCNWbnjna.jpg,2021-03-08,Evangelion: 3.0+1.0 Thrice Upon a Time,False,8.6,409
9,False,/l5K9elugftlcyIHHm4nylvsn26X.jpg,[18],255709,ko,소원,After 8-year-old So-won narrowly survives a br...,9.923,/x9yjkm9gIz5qI5fJMUTfBnWiB2o.jpg,2013-10-02,Hope,False,8.5,238


In [161]:
temp.shape

(9420, 14)

### filtering the columns required

In [162]:
movietab = temp[['title','overview','genre_ids']]

In [163]:
movietab

,title,overview,genre_ids
0,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[35, 18, 10749]"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[18, 80]"
2,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]"
3,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,[10749]
4,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]"
...,...,...,...
15,Piranha 3DD,"After the events at Lake Victoria, the prehist...","[27, 53, 35]"
16,Baise-moi,Manu has lived a difficult life. Abused and vi...,"[80, 18, 10749, 53]"
17,Batman & Robin,Batman and Robin deal with relationship issues...,"[878, 28, 14]"
18,Melissa P.,"An adolescent girl, living with her mother and...",[18]


### Getting the genre data

In [164]:
## read genre data
url = "https://api.themoviedb.org/3/genre/movie/list?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US"
response = requests.get(url)

In [165]:
response.json()['genres']

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 80, 'name': 'Crime'},
 {'id': 99, 'name': 'Documentary'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 36, 'name': 'History'},
 {'id': 27, 'name': 'Horror'},
 {'id': 10402, 'name': 'Music'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 10770, 'name': 'TV Movie'},
 {'id': 53, 'name': 'Thriller'},
 {'id': 10752, 'name': 'War'},
 {'id': 37, 'name': 'Western'}]

In [166]:
gen = pd.DataFrame(response.json()['genres'])

In [167]:
gen

,id,name
0,28,Action
1,12,Adventure
2,16,Animation
3,35,Comedy
4,80,Crime
5,99,Documentary
6,18,Drama
7,10751,Family
8,14,Fantasy
9,36,History


### zipping the dict values to one dictionary

In [168]:
genr_dict = dict(zip(gen.id, gen.name))

In [169]:
genr_dict

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [170]:
def update_genre(row_val):
    val = []
    for each_ele in row_val:
        if each_ele in genr_dict:
            val.append(genr_dict[each_ele])
    return ','.join(str(e) for e in val)        

In [171]:
movietab['genre'] = movietab['genre_ids'].apply(update_genre)

<ipython-input-171-58283d34c2b8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movietab['genre'] = movietab['genre_ids'].apply(update_genre)


In [172]:
movietab

,title,overview,genre_ids,genre
0,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","[35, 18, 10749]","Comedy,Drama,Romance"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"[18, 80]","Drama,Crime"
2,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","[18, 80]","Drama,Crime"
3,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,[10749],Romance
4,Schindler's List,The true story of how businessman Oskar Schind...,"[18, 36, 10752]","Drama,History,War"
...,...,...,...,...
15,Piranha 3DD,"After the events at Lake Victoria, the prehist...","[27, 53, 35]","Horror,Thriller,Comedy"
16,Baise-moi,Manu has lived a difficult life. Abused and vi...,"[80, 18, 10749, 53]","Crime,Drama,Romance,Thriller"
17,Batman & Robin,Batman and Robin deal with relationship issues...,"[878, 28, 14]","Science Fiction,Action,Fantasy"
18,Melissa P.,"An adolescent girl, living with her mother and...",[18],Drama


In [173]:
movietab.drop(['genre_ids'],axis = 1,inplace=True)

C:\Users\bhanu\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [174]:
movietab

,title,overview,genre
0,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Comedy,Drama,Romance"
1,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,"Drama,Crime"
2,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Drama,Crime"
3,Gabriel's Inferno Part II,Professor Gabriel Emerson finally learns the t...,Romance
4,Schindler's List,The true story of how businessman Oskar Schind...,"Drama,History,War"
...,...,...,...
15,Piranha 3DD,"After the events at Lake Victoria, the prehist...","Horror,Thriller,Comedy"
16,Baise-moi,Manu has lived a difficult life. Abused and vi...,"Crime,Drama,Romance,Thriller"
17,Batman & Robin,Batman and Robin deal with relationship issues...,"Science Fiction,Action,Fantasy"
18,Melissa P.,"An adolescent girl, living with her mother and...",Drama


In [175]:
movietab.to_csv("final_movie_data.csv")

## Text Preprocesing on overview column

### 1. Lowercasing

In [226]:
movietab['overview'][1]

1    framed in the 1940s for the double murder of h...
1    in a time of superstition and magic when wolve...
1    buscapé was raised in a very violent environme...
1    two years after bringing peace to the galaxy s...
1    léon the top hit man in new york has earned a ...
                           ...                        
1    in ancient greece 1200 bc a queen succumbs to ...
1    two teenage yoga enthusiasts team up with a le...
1    the new installment of the sharknado franchise...
1    a drama centered on a troubled young woman who...
1    a young nurse takes care of elderly author who...
Name: overview, Length: 471, dtype: object

In [227]:
movietab['overview'] = movietab['overview'].str.lower()

<ipython-input-227-722d882d95a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movietab['overview'] = movietab['overview'].str.lower()


In [228]:
movietab

,title,overview,genre
0,Dilwale Dulhania Le Jayenge,raj is a rich carefree happygolucky second gen...,"Comedy,Drama,Romance"
1,The Shawshank Redemption,framed in the 1940s for the double murder of h...,"Drama,Crime"
2,The Godfather,spanning the years 1945 to 1955 a chronicle of...,"Drama,Crime"
3,Gabriel's Inferno Part II,professor gabriel emerson finally learns the t...,Romance
4,Schindler's List,the true story of how businessman oskar schind...,"Drama,History,War"
...,...,...,...
15,Piranha 3DD,after the events at lake victoria the prehisto...,"Horror,Thriller,Comedy"
16,Baise-moi,manu has lived a difficult life abused and vio...,"Crime,Drama,Romance,Thriller"
17,Batman & Robin,batman and robin deal with relationship issues...,"Science Fiction,Action,Fantasy"
18,Melissa P.,an adolescent girl living with her mother and ...,Drama


## 2. Removing Panctuations

###### Python punctioation list ->  # !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

In [229]:
import string
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [230]:
exclude = string.punctuation


In [231]:
def remove_punc(text):
    return text.translate(str.maketrans('', '', exclude))

In [232]:
movietab['overview'] = movietab['overview'].apply(remove_punc)

<ipython-input-232-2a560deb6011>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movietab['overview'] = movietab['overview'].apply(remove_punc)


In [233]:
movietab

,title,overview,genre
0,Dilwale Dulhania Le Jayenge,raj is a rich carefree happygolucky second gen...,"Comedy,Drama,Romance"
1,The Shawshank Redemption,framed in the 1940s for the double murder of h...,"Drama,Crime"
2,The Godfather,spanning the years 1945 to 1955 a chronicle of...,"Drama,Crime"
3,Gabriel's Inferno Part II,professor gabriel emerson finally learns the t...,Romance
4,Schindler's List,the true story of how businessman oskar schind...,"Drama,History,War"
...,...,...,...
15,Piranha 3DD,after the events at lake victoria the prehisto...,"Horror,Thriller,Comedy"
16,Baise-moi,manu has lived a difficult life abused and vio...,"Crime,Drama,Romance,Thriller"
17,Batman & Robin,batman and robin deal with relationship issues...,"Science Fiction,Action,Fantasy"
18,Melissa P.,an adolescent girl living with her mother and ...,Drama


## 3. Spelling Correction using TextBlob

In [234]:
from textblob import TextBlob

In [235]:
movietab['overview'] = movietab.overview.apply(lambda txt: ''.join(TextBlob(txt).correct()))

<ipython-input-235-27d972d35847>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movietab['overview'] = movietab.overview.apply(lambda txt: ''.join(TextBlob(txt).correct()))


In [211]:
movietab

,title,overview,genre
0,Dilwale Dulhania Le Jayenge,raj is a rich carefree happygolucky second gen...,"Comedy,Drama,Romance"
1,The Shawshank Redemption,framed in the 1940s for the double murder of h...,"Drama,Crime"
2,The Godfather,spanning the years 1945 to 1955 a chronicle of...,"Drama,Crime"
3,Gabriel's Inferno Part II,professor gabriel emerson finally learns the t...,Romance
4,Schindler's List,the true story of how businessman oskar schind...,"Drama,History,War"
...,...,...,...
15,Piranha 3DD,after the events at lake victoria the prehisto...,"Horror,Thriller,Comedy"
16,Baise-moi,manu has lived a difficult life abused and vio...,"Crime,Drama,Romance,Thriller"
17,Batman & Robin,batman and robin deal with relationship issues...,"Science Fiction,Action,Fantasy"
18,Melissa P.,an adolescent girl living with her mother and ...,Drama


## 4. Removing stop words

In [236]:
from nltk.corpus import stopwords

In [238]:
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
   # new_text.clear()
    return " ".join(x)

In [239]:
movietab['overview'] = movietab['overview'].apply(remove_stopwords)

<ipython-input-239-b726988392b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movietab['overview'] = movietab['overview'].apply(remove_stopwords)


## 5 Tokenization

In [240]:
from nltk.tokenize import word_tokenize

In [241]:
movietab['overview'] = movietab['overview'].apply(word_tokenize)

<ipython-input-241-de676d4296be>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movietab['overview'] = movietab['overview'].apply(word_tokenize)


In [249]:
movietab

,title,overview,genre
0,Dilwale Dulhania Le Jayenge,"[ran, rich, carefree, happygolucky, second, ge...","Comedy,Drama,Romance"
1,The Shawshank Redemption,"[framed, 1940s, double, murder, wife, lover, s...","Drama,Crime"
2,The Godfather,"[spanning, years, 1945, 1955, chronicle, secti...","Drama,Crime"
3,Gabriel's Inferno Part II,"[professor, gabriel, emerson, finally, learns,...",Romance
4,Schindler's List,"[true, story, businessman, oscar, schindler, s...","Drama,History,War"
...,...,...,...
15,Piranha 3DD,"[events, lake, victoria, prehistoric, school, ...","Horror,Thriller,Comedy"
16,Baise-moi,"[man, lived, difficult, life, abused, violentl...","Crime,Drama,Romance,Thriller"
17,Batman & Robin,"[batman, robin, deal, relationship, issues, pr...","Science Fiction,Action,Fantasy"
18,Melissa P.,"[adolescent, girl, living, mother, grandmother...",Drama


## 6 Stemming

In [242]:
from nltk.stem.porter import PorterStemmer

In [250]:
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text])

In [251]:
movietab['overview'] = movietab['overview'].apply(stem_words)

<ipython-input-251-faa841ed8c7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movietab['overview'] = movietab['overview'].apply(stem_words)


In [252]:
movietab

,title,overview,genre
0,Dilwale Dulhania Le Jayenge,ran rich carefre happygolucki second gener ni ...,"Comedy,Drama,Romance"
1,The Shawshank Redemption,frame 1940 doubl murder wife lover stand banke...,"Drama,Crime"
2,The Godfather,span year 1945 1955 chronicl section italianam...,"Drama,Crime"
3,Gabriel's Inferno Part II,professor gabriel emerson final learn truth ju...,Romance
4,Schindler's List,true stori businessman oscar schindler save th...,"Drama,History,War"
...,...,...,...
15,Piranha 3DD,event lake victoria prehistor school bloodthir...,"Horror,Thriller,Comedy"
16,Baise-moi,man live difficult life abus violent rapid set...,"Crime,Drama,Romance,Thriller"
17,Batman & Robin,batman robin deal relationship issu prevent mr...,"Science Fiction,Action,Fantasy"
18,Melissa P.,adolesc girl live mother grandmoth first sexua...,Drama
